# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [5]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

import joblib

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136636
aml-quickstarts-136636
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [3]:
# Create compute cluster and choose a name for it
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-30T19:45:26.993000+00:00', 'errors': None, 'creationTime': '2021-01-30T18:48:51.881735+00:00', 'modifiedTime': '2021-01-30T18:49:07.652577+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

data = pd.read_csv('./heart_failure_clinical_records_dataset.csv')

found = False
key = "heart-failure-prediction"
description_text = "Prediction of patients’ survival with heart failure - Capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        my_dataset = 'https://github.com/dimikara/heart-failure-prediction/blob/master/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
                                
# Preview of the first five rows
print(data.head())

# Explore data
print(data.describe())

df = dataset.to_pandas_dataframe()
df.describe()

# Data columns
df.columns = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time', 'DEATH_EVENT']
x = df[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']]
y = df[['DEATH_EVENT']]

experiment_name = 'heart-failure-prediction'
project_folder = './capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

run = experiment.start_logging()


quick-starts-ws-136636
aml-quickstarts-136636
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77
    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smo

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Specify parameter sampler 
# Create the different params that you will be using during training

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=compute_target,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [8]:
# Submit the hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2
Web View: https://ml.azure.com/experiments/heart-failure-prediction/runs/HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-136636/workspaces/quick-starts-ws-136636

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-30T19:49:12.698666][API][INFO]Experiment created<END>\n""<START>[2021-01-30T19:49:13.074433][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-30T19:49:13.228287][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-30T19:49:13.8412350Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2
Web View: https://ml.azure.com/experiments/heart-failure-prediction/runs/HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2?wsid=/s

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))


[{'run_id': 'HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2_12', 'hyperparameters': '{"--C": 500, "--max_iter": 300}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2_8', 'hyperparameters': '{"--C": 500, "--max_iter": 200}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2_7', 'hyperparameters': '{"--C": 1, "--max_iter": 200}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2_2', 'hyperparameters': '{"--C": 500, "--max_iter": 50}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2_3', 'hyperparameters': '{"--C": 0.01, "--max_iter": 50}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2_1', 'hyperparameters': '{"--C": 100, "--max_iter": 200}', 'best_

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
# Get the best run and save the model from that run.

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

# get_file_names()
# Returns a list of the files that are stored in association with the run.

print("Best run file names :",best_run.get_file_names())

# log_accuracy_table()
# Logs an accuracy table to the artifact store. Maybe use it for testing.
# log_accuracy_table(name, value, description='')


Best run metrics : {'Regularization Strength:': 100.0, 'Max iterations:': 200, 'Accuracy': 0.8333333333333334}
Best run details : {'runId': 'HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2_1', 'target': 'compute-cluster', 'status': 'Completed', 'startTimeUtc': '2021-01-30T19:53:18.646001Z', 'endTimeUtc': '2021-01-30T19:55:09.650346Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '302c0a18-9fa0-4a0f-99ca-a5eba06a1056', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '100', '--max_iter', '200'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'compute-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'priority': None, 'credentialPassthrough

In [11]:
#TODO: Save the best model

# filename = 'azureml_best_model.sav'
# joblib.dump(best_run,filename)


# best_run, best_model = run.get_output()
best_run.register_model(model_name = "best_run_hyperdrive.pkl", model_path = './outputs/')
# print(best_run._final_estimator)
print(best_run)


Run(Experiment: heart-failure-prediction,
Id: HD_e8d3375c-9f5e-430a-ba03-2a6aedab15a2_1,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service